In [1]:
from Core.filter import RdrFilter
from Core.mapper import PyGMT, save_qgis_layer

explorer = RdrFilter()
pygmt = PyGMT()

RDRCUMINDEX.TAB already exists. Set `reload = True` to force download.
RDRCUMINDEX.LBL already exists. Set `reload = True` to force download.


In [2]:
explorer.load_df()

NO FILTER ............................................. 93425 images


In [3]:
# df = explorer.latitude_filter(commit=False)
# pygmt.show_on_map(df, target = 'img_rectangle', filename = 'latitude_flt')
# save_qgis_layer(df, target = 'img_rectangle', filename = 'latitude_flt_rect')

explorer.latitude_filter()

LATITUDE FILTER ........................................ 5904 images


In [4]:
# df = explorer.scale_filter(commit = False)
# pygmt.show_on_map(df, target = 'img_rectangle', filename = 'scale_flt')
explorer.scale_filter()

SCALE FILTER ........................................... 3699 images


In [5]:
# df = explorer.season_filter(commit = False)
# pygmt.show_on_map(df, target = 'img_rectangle', filename = 'season_flt')
explorer.season_filter()

SEASON FILTER .......................................... 2081 images


In [6]:
# df = explorer.cluster_filter(commit = False)
# pygmt.show_on_map(df, target = 'img_rectangle', filename = 'cluster_flt')
explorer.cluster_filter()

CLUSTER FILTER ......................................... 1503 images


In [7]:
# df = explorer.keyword_filter(commit = False)
# pygmt.show_on_map(df, target = 'img_rectangle', filename = 'keyword_flt')
explorer.keyword_filter()

KEYWORD FILTER .......................................... 354 images


In [8]:
# df = explorer.my_filter(commit = False)
# pygmt.show_on_map(df, target = 'img_rectangle', filename = 'my_flt')
explorer.my_filter()

MY FILTER ............................................... 334 images


In [9]:
df = explorer.allignment_filter(commit = False)
save_qgis_layer(df, target = 'img_rectangle', filename = 'allignment_flt')
# pygmt.show_on_map(df, target = 'img_rectangle', filename = 'allignment_flt')
# explorer.allignment_filter()

ALLIGNMENT FILTER ....................................... 203 images


In [10]:
explorer.local_conf

{'latitude_filter': {'min_lat': 78},
 'scale_filter': {'scale': 0.25},
 'season_filter': {'season': 'Northern summer'},
 'cluster_filter': {'algorithm': 'dbscan', 'min_samples': 2, 'epsilon': 2000},
 'keyword_filter': {'keywords': ['scarp']},
 'my_filter': {'min_years': 2, 'mys': [], 'consecutive': False},
 'allignment_filter': {}}

In [ ]:
# import os
# import sys

# # Set environment variables so QGIS can find its libraries and plugins
# os.environ['QGIS_PREFIX_PATH'] = 'C:/OSGeo4W/apps/qgis-ltr'
# os.environ['QT_QPA_PLATFORM_PLUGIN_PATH'] = 'C:/OSGeo4W/apps/Qt5/plugins'

# # Prepend OSGeo4W directories to the PATH so DLLs can be found
# os.environ['PATH'] = 'C:/OSGeo4W/bin;' + os.environ['PATH']
# os.environ['PATH'] = 'C:/OSGeo4W/apps/Qt5/bin;' + os.environ['PATH']
# os.environ['PATH'] = 'C:/OSGeo4W/apps/qgis-ltr/bin;' + os.environ['PATH']

# # Add the QGIS Python directory to the Python path
# sys.path.append('C:/OSGeo4W/apps/qgis-ltr/python')
# sys.path.append('C:/OSGeo4W/apps/Python312/Lib/site-packages')

# # Optionally, if you use QGIS plugins:
# sys.path.append('C:/OSGeo4W/apps/qgis-ltr/python/plugins')